## One-hot Encoding

In [1]:
from konlpy.tag import Komoran
import numpy as np

komoran = Komoran()
text = "오늘 날씨는 구름이 많아요."

# 명사만 추출
nouns = komoran.nouns(text)
print(nouns)

['오늘', '날씨', '구름']


In [3]:
# 단어 사전 구축 및 단어별 인덱스 부여, 정수화
dics = {}
for word in nouns: # ['오늘', '날씨', '구름']
    if word not in dics.keys():
        print('dics.keys():', dics.keys())
        print('len(dics):', len(dics))
        # dics dictionary에 word란 키로 dictionary 길이 할당
        dics[word] = len(dics) # dics['오늘'] = 0
        
print(dics)
print(list(dics.values())) # 값만 출력

dics.keys(): dict_keys([])
len(dics): 0
dics.keys(): dict_keys(['오늘'])
len(dics): 1
dics.keys(): dict_keys(['오늘', '날씨'])
len(dics): 2
{'오늘': 0, '날씨': 1, '구름': 2}
[0, 1, 2]


In [8]:
# 원-핫 인코딩
nb_classes = len(dics)        # dictionary 길이
print(type(np.eye(nb_classes)))
print(np.eye(nb_classes))

<class 'numpy.ndarray'>
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


In [ ]:
targets = list(dics.values()) # [0, 1, 2]
one_hot_targets = np.eye(nb_classes)[targets]
print(one_hot_targets)
print()
one_hot_targets = np.eye(nb_classes)[[2, 1, 1]]
print(one_hot_targets)

### Word2Vec

In [1]:
line = "8112052	어릴때보고 지금다시봐도 재밌어요ㅋㅋ	1"
print(line.split('\t'))
line = "8132799	디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산업이 부러웠는데. 사실 우리나라에서도 그 어려운시절에 끝까지 열정을 지킨 노라노 같은 전통이있어 저와 같은 사람들이 꿈을 꾸고 이뤄나갈 수 있다는 것에 감사합니다.	1"
print(line.split('\t'))

['8112052', '어릴때보고 지금다시봐도 재밌어요ㅋㅋ', '1']
['8132799', '디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산업이 부러웠는데. 사실 우리나라에서도 그 어려운시절에 끝까지 열정을 지킨 노라노 같은 전통이있어 저와 같은 사람들이 꿈을 꾸고 이뤄나갈 수 있다는 것에 감사합니다.', '1']


In [2]:
from gensim.models import Word2Vec
from konlpy.tag import Komoran
import time


# 네이버 영화 리뷰 데이터 읽어옴
def read_review_data(filename):
    with open(filename, 'r', encoding='utf-8') as f:
        data = [line.split('\t') for line in f.read().splitlines()]
        data = data[1:] # header 제거
    return data # 2차원 list

In [3]:
# 측정 시작
start = time.time()

# 리뷰 파일 읽어오기, 말뭉치: 문장의 모임, 분석 대상의 전체
print('1) 말뭉치 데이터 읽기 시작')
review_data = read_review_data('./data/ratings.txt')
print('2) 리뷰 데이터 전체 개수:', len(review_data)) # 리뷰 데이터 전체 개수
print('3) 말뭉치 데이터 읽기 완료: ', time.time() - start)
print(review_data[0:5])

1) 말뭉치 데이터 읽기 시작
2) 리뷰 데이터 전체 개수: 200000
3) 말뭉치 데이터 읽기 완료:  0.5968971252441406
[['8112052', '어릴때보고 지금다시봐도 재밌어요ㅋㅋ', '1'], ['8132799', '디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산업이 부러웠는데. 사실 우리나라에서도 그 어려운시절에 끝까지 열정을 지킨 노라노 같은 전통이있어 저와 같은 사람들이 꿈을 꾸고 이뤄나갈 수 있다는 것에 감사합니다.', '1'], ['4655635', '폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.', '1'], ['9251303', '와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런게 진짜 영화지', '1'], ['10067386', '안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.', '1']]


In [4]:
print([sentence[0] for sentence in review_data[0:2]]) # 첫번째 변수
print()
print([sentence[1] for sentence in review_data[0:2]])

['8112052', '8132799']

['어릴때보고 지금다시봐도 재밌어요ㅋㅋ', '디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산업이 부러웠는데. 사실 우리나라에서도 그 어려운시절에 끝까지 열정을 지킨 노라노 같은 전통이있어 저와 같은 사람들이 꿈을 꾸고 이뤄나갈 수 있다는 것에 감사합니다.']


In [5]:
start = time.time()

# 문장단위로 명사만 추출해 학습 입력 데이터로 만듬
print('2) 형태소에서 명사만 추출 시작')
komoran = Komoran() # 형태소 분석기의 객체 생성
# sentence[1]: document, 리뷰, 시간이 많이 필요함 core 6: 85초, i5 2320: 177
docs = [komoran.nouns(sentence[1]) for sentence in review_data]
print('2) 형태소에서 명사만 추출 완료: ', time.time() - start)
print(docs[0:2])

2) 형태소에서 명사만 추출 시작
2) 형태소에서 명사만 추출 완료:  177.18568515777588
[['때'], ['디자인', '학생', '외국', '디자이너', '전통', '발전', '문화', '산업', '사실', '우리나라', '시절', '끝', '열정', '노라', '노', '전통', '사람', '꿈', '수', '것', '감사']]


In [7]:
# import pickle # 파이썬 자료형 저장 지원
import joblib # 병렬 처리 지원, 모델 저장, 로딩

joblib.dump(docs, './ratings_docs.pkl') 

['./ratings_docs.pkl']

In [8]:
docs2 = joblib.load('./ratings_docs.pkl') 
print(docs2[0:2])

[['때'], ['디자인', '학생', '외국', '디자이너', '전통', '발전', '문화', '산업', '사실', '우리나라', '시절', '끝', '열정', '노라', '노', '전통', '사람', '꿈', '수', '것', '감사']]


In [9]:
start = time.time()

# word2vec 모델 학습
print('3) word2vec 모델 학습 시작')
# sentences=docs: 명사 토큰의 집합
# vector_size=200: 200개의 변수로 docs에 존재하는 토큰을 수치(벡터)화 함 
# window=4: 주변단어 4개까지 확인
# min_count=2: 빈도수 2이하의 단어들은 학습하지 않음.
# sg: 0(CBOW 모델), 1(skip-gram 모델)
model = Word2Vec(sentences=docs, vector_size=200, window=4, min_count=2, sg=1)
print('3) word2vec 모델 학습 완료: ', time.time() - start)

3) word2vec 모델 학습 시작
3) word2vec 모델 학습 완료:  15.55300784111023


In [10]:
start = time.time()

# 모델 저장
print('4) 학습된 모델 저장 시작')
model.save('./ratings.model')
print('4) 학습된 모델 저장 완료: ', time.time() - start)

4) 학습된 모델 저장 시작
4) 학습된 모델 저장 완료:  0.07805061340332031


In [11]:
start = time.time()

# 학습된 말뭉치 개수, 말뭉치(코퍼스) 내 전체 단어 개수
print("corpus_count : ", model.corpus_count) # 문장수
print("corpus_total_words : ", model.corpus_total_words) # 단어수

corpus_count :  200000
corpus_total_words :  1076896


In [13]:
# 모델 로딩
model = Word2Vec.load('./ratings.model')
print("corpus_total_words : ", model.corpus_total_words)

# '사랑'이란 단어로 생성한 단어 임베딩 벡터, 200개의 실수값으로 '사랑' 표현
print('일요일 : ', len(model.wv['일요일']), model.wv['일요일'])

corpus_total_words :  1076896
일요일 :  200 [ 0.09601194 -0.06301533  0.05377556  0.05146767  0.08600172 -0.18279502
  0.11715929  0.23092006  0.02606846  0.07284421  0.02399201 -0.2582403
  0.1249833   0.05983743  0.09475879  0.10584436 -0.14510472 -0.1474172
  0.11248036 -0.08936764  0.2642022  -0.09662985 -0.13528572  0.09430071
  0.07115527 -0.05240937  0.06333485  0.04684192 -0.10860016 -0.17254148
  0.18870533  0.01000663  0.02182337 -0.10107178  0.02372091 -0.01131875
  0.26405787  0.06825031 -0.07916279 -0.02596795 -0.10694709 -0.06646781
 -0.01669144  0.0707752   0.07875628  0.05340417 -0.17502826  0.00176028
  0.19976006  0.16485247  0.05665494 -0.02617382 -0.10000215  0.10478491
  0.02070545  0.15489574  0.1910311  -0.09852612 -0.00601866  0.11823366
  0.05474098  0.0406172   0.1503287   0.1298537  -0.1972055   0.0092661
 -0.10677249  0.10701106 -0.11345098  0.11067979 -0.01895295 -0.00505385
  0.26622763 -0.01686645 -0.07012222 -0.01199859  0.07126155 -0.16521606
 -0.24455792 

In [21]:
# 단어 유사도 계산, 1에 가까울수록 유사도가 높음
# 좋은 말뭉치 모델을 사용시 언어 학습이 완성도있게 이루어짐.
print("일요일 = 월요일\t", model.wv.similarity(w1='일요일', w2='월요일'))
print("안성기 = 배우\t", model.wv.similarity(w1='안성기', w2='배우'))
print("눈물 = 웃음\t", model.wv.similarity(w1='눈물', w2='웃음'))
print("배우 = 연기\t", model.wv.similarity(w1='배우', w2='연기'))
print("명동 = 쇼핑\t", model.wv.similarity(w1='명동', w2='쇼핑'))

일요일 = 월요일	 0.8930642
안성기 = 배우	 0.719242
눈물 = 웃음	 0.6519672
배우 = 연기	 0.61714625
명동 = 쇼핑	 0.92532635


In [22]:
# 가장 유사한 단어 추출
print(model.wv.most_similar("안성기", topn=5))
print(model.wv.most_similar("시리즈", topn=5))

[('한석규', 0.9403318762779236), ('김수로', 0.9400565028190613), ('정재영', 0.9369944334030151), ('최민식', 0.9352214932441711), ('지진희', 0.9331547617912292)]
[('엑스맨', 0.8179543018341064), ('스타워즈', 0.8018988370895386), ('포터', 0.7916746735572815), ('다이하드', 0.790935218334198), ('반지의 제왕', 0.7854950428009033)]
